In [ ]:
from langchain_anthropic import ChatAnthropic
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.prompts import HumanMessagePromptTemplate, SystemMessagePromptTemplate
import os

# Set your API key
os.environ["ANTHROPIC_API_KEY"] = "your-api-key-here"

# Initialize Claude
llm = ChatAnthropic(
    model="claude-sonnet-4-5-20250929",
    temperature=0.7,
    max_tokens=1024
)

# ============================================================
# Example 1: Simple Prompt Template
# ============================================================
print("Example 1: Simple Prompt Template")
print("-" * 60)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "Tell me about {topic}")
])

chain = prompt | llm

response = chain.invoke({"topic": "machine learning"})
print(response.content)
print("\n")

# ============================================================
# Example 2: Multiple Variables
# ============================================================
print("Example 2: Multiple Variables")
print("-" * 60)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert {role}."),
    ("human", "Explain {concept} to a {audience}")
])

chain = prompt | llm

response = chain.invoke({
    "role": "data scientist",
    "concept": "neural networks",
    "audience": "beginner"
})
print(response.content)
print("\n")

# ============================================================
# Example 3: Reusable Template for Multiple Queries
# ============================================================
print("Example 3: Reusable Template")
print("-" * 60)

# Create a reusable template for product descriptions
product_template = ChatPromptTemplate.from_messages([
    ("system", "You are a creative marketing copywriter."),
    ("human", "Write a compelling product description for {product_name}. Key features: {features}")
])

chain = product_template | llm

# Use the same template for different products
products = [
    {"product_name": "Smart Watch", "features": "heart rate monitor, GPS, waterproof"},
    {"product_name": "Wireless Earbuds", "features": "noise cancellation, 24hr battery, touch controls"}
]

for product in products:
    response = chain.invoke(product)
    print(f"\n{product['product_name']}:")
    print(response.content)
print("\n")

# ============================================================
# Example 4: Template with Few-Shot Examples
# ============================================================
print("Example 4: Few-Shot Learning Template")
print("-" * 60)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a sentiment analyzer. Classify text as Positive, Negative, or Neutral."),
    ("human", "Text: 'I love this product!' -> Sentiment: Positive"),
    ("human", "Text: 'This is terrible.' -> Sentiment: Negative"),
    ("human", "Text: 'It's okay.' -> Sentiment: Neutral"),
    ("human", "Text: '{text}' -> Sentiment:")
])

chain = prompt | llm

texts = [
    "This exceeded my expectations!",
    "Waste of money",
    "It works as described"
]

for text in texts:
    response = chain.invoke({"text": text})
    print(f"Text: '{text}'")
    print(f"Sentiment: {response.content}\n")

# ============================================================
# Example 5: Complex Template with Formatting
# ============================================================
print("Example 5: Complex Structured Template")
print("-" * 60)

code_review_template = ChatPromptTemplate.from_messages([
    ("system", """You are a senior software engineer doing code reviews.
    Provide feedback in this format:
    1. Overall Assessment
    2. Issues Found
    3. Suggestions for Improvement
    4. Best Practices"""),
    ("human", """Review this {language} code:
    
Code:
{code}

Focus on: {focus_areas}""")
])

chain = code_review_template | llm

response = chain.invoke({
    "language": "Python",
    "code": """
def calculate(a, b):
    return a + b
result = calculate(5, 3)
print(result)
    """,
    "focus_areas": "error handling, code quality, and documentation"
})
print(response.content)
print("\n")

# ============================================================
# Example 6: Template with Conditional Logic
# ============================================================
print("Example 6: Template Builder with Conditions")
print("-" * 60)

def build_prompt(include_examples=True, difficulty="beginner"):
    messages = [
        ("system", f"You are a {difficulty}-level programming tutor.")
    ]
    
    if include_examples:
        messages.append(("human", "Always provide code examples in your explanations."))
    
    messages.append(("human", "Explain: {topic}"))
    
    return ChatPromptTemplate.from_messages(messages)

# Use with examples
prompt_with_examples = build_prompt(include_examples=True, difficulty="advanced")
chain = prompt_with_examples | llm
response = chain.invoke({"topic": "decorators in Python"})
print("With examples:")
print(response.content)
print("\n")

# ============================================================
# Example 7: Template for Data Extraction
# ============================================================
print("Example 7: Data Extraction Template")
print("-" * 60)

extraction_template = ChatPromptTemplate.from_messages([
    ("system", "Extract structured information from text. Return as JSON format."),
    ("human", """Extract the following from this text:
- Name
- Email
- Phone
- Company

Text: {text}""")
])

chain = extraction_template | llm

response = chain.invoke({
    "text": "Contact John Smith at john.smith@email.com or call 555-1234. He works at Tech Corp."
})
print(response.content)
print("\n")

# ============================================================
# Example 8: Chaining Multiple Templates
# ============================================================
print("Example 8: Multi-Step Template Chain")
print("-" * 60)

# Step 1: Generate ideas
idea_template = ChatPromptTemplate.from_messages([
    ("system", "You are a creative brainstorming assistant."),
    ("human", "Generate 3 {content_type} ideas about {topic}. Just list the titles.")
])

# Step 2: Expand on one idea
expansion_template = ChatPromptTemplate.from_messages([
    ("system", "You are a content writer."),
    ("human", "Expand on this {content_type} idea in 2-3 sentences: {idea}")
])

# Execute the chain
idea_chain = idea_template | llm
ideas_response = idea_chain.invoke({
    "content_type": "blog post",
    "topic": "artificial intelligence"
})
print("Generated Ideas:")
print(ideas_response.content)

# Pick first idea and expand
first_idea = ideas_response.content.split('\n')[0]
expansion_chain = expansion_template | llm
expanded = expansion_chain.invoke({
    "content_type": "blog post",
    "idea": first_idea
})
print(f"\nExpanded Idea:")
print(expanded.content)
print("\n")

# ============================================================
# Example 9: Template with Partial Variables
# ============================================================
print("Example 9: Partial Variables")
print("-" * 60)

# Create a template with some pre-filled values
base_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a {role} who specializes in {specialty}."),
    ("human", "{question}")
])

# Create a specialized version with partial values
math_tutor_prompt = base_prompt.partial(
    role="mathematics tutor",
    specialty="algebra and calculus"
)

chain = math_tutor_prompt | llm
response = chain.invoke({"question": "What is a limit?"})
print(response.content)
print("\n")

# ============================================================
# Example 10: Template Library Pattern
# ============================================================
print("Example 10: Template Library")
print("-" * 60)

class TemplateLibrary:
    """Reusable template library for common tasks"""
    
    @staticmethod
    def translator(source_lang, target_lang):
        return ChatPromptTemplate.from_messages([
            ("system", f"You are a translator. Translate from {source_lang} to {target_lang}."),
            ("human", "{text}")
        ])
    
    @staticmethod
    def summarizer(max_words):
        return ChatPromptTemplate.from_messages([
            ("system", f"Summarize the following text in under {max_words} words."),
            ("human", "{text}")
        ])
    
    @staticmethod
    def explainer(audience):
        return ChatPromptTemplate.from_messages([
            ("system", f"Explain concepts for a {audience} audience."),
            ("human", "Explain: {concept}")
        ])

# Use the library
library = TemplateLibrary()

# Translation
translate_chain = library.translator("English", "Spanish") | llm
result = translate_chain.invoke({"text": "Hello, how are you?"})
print(f"Translation: {result.content}")

# Summarization
summary_chain = library.summarizer(50) | llm
result = summary_chain.invoke({
    "text": "Artificial intelligence is transforming industries worldwide. Machine learning algorithms can now perform tasks that once required human intelligence. This technology is being applied in healthcare, finance, transportation, and many other sectors."
})
print(f"\nSummary: {result.content}")

# Explanation
explain_chain = library.explainer("5-year-old") | llm
result = explain_chain.invoke({"concept": "gravity"})
print(f"\nExplanation: {result.content}")